# Solución de problemas con regresión polinomial

En este tutorial conocerás cómo crear, entrenar y evaluar un modelo de regresión polinomial para un conjunto de datos de múltiples variables. Adicionalmente, aprenderás a realizar un ajuste de hiperparámetros para este tipo de regresión. En ese sentido, veremos cómo realizar los siguientes procesos:

1. Importar las librerías necesarias.
2. Cargar un conjunto de datos.
3. Preparar los datos para el modelado.
4. Entrenar un modelo de regresión polinomial.
5. Realizar una búsqueda de hiperparámetros.

Utilizaremos el conjunto de datos correspondiente a la caracterización de casas y su precio. Nuestro objetivo es, entonces, obtener un modelo con regresión polinomial para predecir el precio de una vivienda dadas sus características.

## 1. Importación de librerías requeridas

Importaremos las librerías `pandas` y `scikit-learn`. En particular, usaremos las siguientes clases y funciones:

* `GridSearchCV`: clase para entrenar múltiples modelos variando sus parámetros. Se utiliza para hacer una búsqueda exhaustiva de los mejores valores para el entrenamiento de un modelo.
* `KFold`: clase para definir múltiples conjuntos de entrenamiento y validación sobre nuestro conjunto de datos.
* `RobustScaler`: clase para escalar los valores de nuestro conjunto de datos.
* `PolynomialFeatures`: genera un nuevo conjunto de variables que representa todas las combinaciones polinomiales de grado menor o igual a un grado especificado como parámetro.
* `make_pipeline`: función para definir una secuencia de pasos para transformar datos y entrenar modelos.

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import PolynomialFeatures, RobustScaler
from sklearn.pipeline import make_pipeline

## 2. Carga de datos

Realizaremos la carga de datos usando la función de Pandas `read_csv()` y especificando el separador del archivo:

In [2]:
data_raw = pd.read_csv('data/kc_house_data.csv', sep=',')

Veremos los primeros datos del conjunto usando `head()`:

In [3]:
data_raw.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


## 3. Preparación de los datos

Utilizaremos la variable `data` para almacenar un conjunto de datos modificado. En ese sentido, podremos tener una copia de los datos originales almacenada en la variable `data_raw`, por si en algún momento es necesario recuperar información que haya sido modificada:

In [4]:
data = data_raw

### Eliminación de variables poco relevantes

Eliminaremos tres variables con poca relevancia para el precio de las viviendas: `id`, `date` y `zipcode`, haciendo uso de la función `drop()` sobre nuestro DataFrame:

In [5]:
data = data.drop(['id','date','zipcode'], axis=1)

Veremos el resultado con `data.head()`:

In [6]:
data.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15
0,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,47.5112,-122.257,1340,5650
1,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,47.7210,-122.319,1690,7639
2,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,47.7379,-122.233,2720,8062
3,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,47.5208,-122.393,1360,5000
4,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,47.6168,-122.045,1800,7503


### División de datos

Vamos a separar la variable objetivo y las variables independientes para, posteriormente, crear los conjuntos de entrenamiento y pruebas:

In [7]:
x = data.drop(['price'], axis=1)
y = data['price']

Verificaremos que nuestro conjunto `x` no contiene la variable `price()` usando `x.head()`:

In [8]:
x.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15
0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,47.5112,-122.257,1340,5650
1,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,47.7210,-122.319,1690,7639
2,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,47.7379,-122.233,2720,8062
3,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,47.5208,-122.393,1360,5000
4,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,47.6168,-122.045,1800,7503


Definiremos los conjuntos de entrenamiento y pruebas usando `train_test_split()`, utilizando el conjunto de variables independientes (`x`) y la variable objetivo (`y`):

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

### Estandarización

Para el caso de regresión polinomial, la escala en la que se encuentran las variables se vuelve relevante, a diferencia de la regresión lineal simple. Específicamente, dado que crearemos nuevas variables combinando las variables originales, es recomendable dejarlas en la misma escala. Es por eso que utilizaremos un objeto de la clase `RobustScaler()` que, por cada variable, elimina la media y escala los valores por cuantiles: 

In [10]:
scaler = RobustScaler()

Este objeto tiene el método `fit_transform()` para realizar la estandarización, pero retorna un arreglo en vez de un DataFrame. Por lo tanto, primero almacenaremos la información de las columnas en la variable `columns` y, finalmente, reconstruiremos el DataFrame:

In [11]:
columns = x_train.columns
x_train = scaler.fit_transform(x_train)
x_train = pd.DataFrame(x_train, columns=columns)

## 4. Entrenamiento de un modelo de regresión polinomial

Con el conjunto de datos preparado, vamos a entrenar un modelo de regresión polinomial multivariable. Inicialmente y como referencia, vamos a definir, entrenar y evaluar un modelo de regresión lineal simple:

### Entrenamiento de un modelo de referencia

Entrenaremos el modelo de regresión lineal sin ninguna modificación, usando los conjuntos de entrenamiento y pruebas que definimos anteriormente:

In [12]:
reg_lineal = LinearRegression().fit(x_train, y_train)

Podemos ver qué coeficiente le corresponde a qué variable independiente usando la función `zip()` y juntando todo en una lista:

In [13]:
list(zip(x_train.columns, reg_lineal.coef_))

[('bedrooms', -33190.940884592295),
 ('bathrooms', 30225.215600927695),
 ('sqft_living', 102641.26961008689),
 ('sqft_lot', 976.6919763080489),
 ('floors', 1809.090177315915),
 ('waterfront', 609402.8097220035),
 ('view', 49405.86787529147),
 ('condition', 30724.06360717471),
 ('grade', 94787.09078382891),
 ('sqft_above', 94998.68702397823),
 ('sqft_basement', 33484.16472796743),
 ('yr_built', -112808.4198273522),
 ('yr_renovated', 21.67728680181766),
 ('lat', 115797.97107795805),
 ('long', -25600.87518872473),
 ('sqft_living15', 25902.760866272372),
 ('sqft_lot15', -2379.776029291402)]

#### Evaluación del modelo

Primero usaremos la variable `scaler` para escalar las variables independientes del conjunto de pruebas:

In [14]:
x_test = scaler.fit_transform(x_test)
x_test = pd.DataFrame(x_test, columns=columns)

Ahora realizaremos predicciones sobre el conjunto de test y obtendremos tres métricas para evaluar el desempeño del modelo de referencia: la raíz del error cuadrático medio (`RMSE`), el error absoluto medio (`MAE`) y el coeficiente de determinación (`R²`):

In [15]:
y_pred = reg_lineal.predict(x_test)

print('------ Modelo de regresión lineal simple----')
print("RMSE: %.2f" % mean_squared_error(y_test, y_pred, squared=False))
print("MAE: %.2f" % mean_absolute_error(y_test, y_pred))
print('R²: %.2f' % r2_score(y_test, y_pred))

------ Modelo de regresión lineal simple----
RMSE: 192947.78
MAE: 123047.62
R²: 0.69


Como puedes observar, el modelo resultante se ajusta un 69% a los datos. Veremos si es posible mejorar el rendimiento utilizando regresión polinomial.

### Entrenamiento de un modelo de regresión polinomial

Teniendo el modelo de referencia, vamos a entrenar un modelo utilizando regresión polinomial. Usaremos los conjuntos de datos separados, pero vamos a incluir un paso adicional antes del entrenamiento, la transformación de datos:

#### Transformación de datos

Asignaremos un objeto de la clase `PolynomialFeatures` a la variable `pf_2`. Esta clase recibe un parámetro, `degree`, que define el grado máximo del polinomio que se quiere construir. Por ejemplo, en este caso queremos construir un polinomio de grado 2, por lo que `PolynomialFeatures` genera un nuevo conjunto de datos con todas las posibles combinaciones de dos o menos variables.

In [16]:
pf_2 = PolynomialFeatures(degree=2)

In [17]:
pf_2

PolynomialFeatures()

Utilizaremos `fit_transform()` para modificar nuestro conjunto de variables independientes, almacenando el resultado en `x_train_pol`:

In [18]:
x_train_pol = pf_2.fit_transform(x_train)

In [19]:
x_train.shape

(17290, 17)

En este conjunto de datos tenemos 17 variables independientes, por lo que usaremos `x_train_pol.shape` para ver cuántos términos se generan:

In [20]:
x_train_pol.shape

(17290, 171)

Como puedes observar, tenemos 171 variables que representan todas las combinaciones polinomiales en un polinomio de grado 2 con 17 variables. Debes tener en cuenta que la complejidad aumenta mucho con respecto al modelo de regresión lineal simple que se entrena solo con 17 variables independientes, incluso con un polinomio de grado bajo. 

#### Entrenamiento del modelo

Ahora entrenaremos el modelo con los datos del conjunto de entrenamiento, almacenando el resultado en la variable `reg_polinomial`:

In [21]:
reg_polinomial = LinearRegression().fit(x_train_pol, y_train)

Para realizar las predicciones, primero vamos a realizar la transformación de los datos del conjunto de pruebas: 

In [22]:
x_test_pol = pf_2.fit_transform(x_test)

Finalmente, podemos utilizar el conjunto de pruebas resultante para evaluar el modelo entrenado:

In [23]:
y_pred = reg_polinomial.predict(x_test_pol)

print('------ Modelo de regresión polinomial múltiple (grado 2)----')
print("RMSE: %.2f" % mean_squared_error(y_test, y_pred, squared=False))
print("MAE: %.2f" % mean_absolute_error(y_test, y_pred))
print('R²: %.2f' % r2_score(y_test, y_pred))

------ Modelo de regresión polinomial múltiple (grado 2)----
RMSE: 163152.26
MAE: 102681.31
R²: 0.78


El modelo de regresión polinomial tiene una mejora considerable en el valor del R<sup>2</sup> (de 0.69 en regresión lineal simple a 0.78 en este modelo) y una mejora también en el valor del RMSE y MAE, lo que indicaría que el modelo tiene un mejor rendimiento. Debes tener en cuenta que el generar combinaciones polinomiales aumenta la complejidad del modelo, es decir, se aumenta el número de variables independientes para realizar el proceso de regresión. Por lo tanto, elegir el grado adecuado es un problema en sí: un grado muy alto puede causar un sobreajuste, pero un grado muy bajo puede no ajustarse lo suficiente a los datos. En este orden de ideas, para este tipo de regresión, el grado del polinomio se convierte en un hiperparámetro que se debe ajustar para obtener un rendimiento óptimo.

Por ejemplo, podemos observar el rendimiento de un modelo de regresión polinomial con grado 3. Crearemos un nuevo objeto de la clase `PolynomialFeatures` y lo almacenaremos en la variable `pf_3`:

In [24]:
pf_3 = PolynomialFeatures(degree=3)

Siguiendo el proceso para el modelo anterior, vamos a transformar el conjunto de entrenamiento:

In [25]:
x_train_pol = pf_3.fit_transform(x_train)

Veremos la cantidad de variables generadas usando `x_train_pol.shape`:

In [26]:
x_train_pol.shape

(17290, 1140)

Como puedes ver, aumentar en uno el valor del hiperparámetro aumenta la cantidad de variables de 171 a 1140, lo que implica un salto considerable de complejidad. Vamos a entrenar el modelo y almacenaremos el resultado en `reg_polinomial`: 

In [27]:
reg_polinomial = LinearRegression().fit(x_train_pol, y_train)

Para evaluar el modelo vamos a transformar los datos del conjunto de pruebas:

In [28]:
x_test_pol = pf_3.fit_transform(x_test)

Finalmente, realizaremos las predicciones:

In [29]:
y_pred = reg_polinomial.predict(x_test_pol)

print('------ Modelo de regresión polinomial múltiple (grado 3)----')
print("RMSE: %.2f" % mean_squared_error(y_test, y_pred, squared=False))
print("MAE: %.2f" % mean_absolute_error(y_test, y_pred))
print('R²: %.2f' % r2_score(y_test, y_pred))

------ Modelo de regresión polinomial múltiple (grado 3)----
RMSE: 322619.38
MAE: 115444.09
R²: 0.12


En este caso, el modelo de regresión polinomial de grado 3 tiene un rendimiento peor, pues se reduce considerablemente el valor de R<sup>2</sup> y se aumentan los valores del RMSE y MAE.

## 5. Búsqueda de hiperparámetros

Como pudiste ver con los modelos anteriores, cambiar el valor de los hiperparámetros tiene un impacto directo sobre el desempeño del modelo resultante, por lo que queremos encontrar un valor que resulte en el mejor desempeño posible. Podemos automatizar el proceso de búsqueda de hiperparámetros mediante la clase `GridSearchCV`, que entrena múltiples modelos y retorna el que tiene mejor desempeño.

Lo primero que haremos es definir la función `PolynomialRegression()` usando la función `make_pipeline()`. Esta función recibe un grado, transforma el conjunto de datos (usando la clase `PolynomialFeatures`) y entrena el modelo de regresión polinomial (usando la clase `LinearRegression()`). Ten en cuenta que esto no siempre es necesario, en este caso se utiliza porque el conjunto de datos cambia al cambiar el valor del hiperparámetro.

In [30]:
def PolynomialRegression(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree), LinearRegression(**kwargs))

Ahora utilizaremos un diccionario para definir nuestro espacio de búsqueda de hiperparámetros, es decir, los valores que vamos a probar y sobre los que decidiremos cuál escoger. Almacenaremos estos valores en la variable `param_grid`, utilizando el nombre `polynomialfeatures__degree`. Este nombre tiene esta forma porque, dentro de la secuencia de pasos que creamos con la función `make_pipeline`, queremos cambiar el parámetro `degree` de la clase `PolynomialFeatures`:

In [31]:
param_grid = {'polynomialfeatures__degree': [2, 3, 4]}

Posteriormente, volveremos a definir un objeto de la clase `KFold()`, que nos será útil para obtener una estimación del desempeño del modelo más realista que simplemente utilizando el conjunto de pruebas. La validación cruzada k-fold es un método que toma el conjunto de entrenamiento original y lo separa en k grupos, usando uno como validación y el resto (k-1) como entrenamiento. Después de definir los grupos, entrena el modelo con el conjunto de entrenamiento y lo evalúa con el conjunto de validación, repitiendo este proceso para cada uno de los k grupos. En este caso, definiremos `k=10` y usaremos el parámetro `shuffle=True` para indicar que se cambie el orden de los datos antes de separarlos en los grupos:

In [32]:
kfold = KFold(n_splits=10, shuffle=True, random_state = 0)

A continuación, vamos a utilizar `GridSearchCV` para realizar una búsqueda exhaustiva de hiperparámetros. Definiremos el algoritmo `PolynomialRegression()`, los valores del hiperparámetro `param_grid` y la estrategia de validación cruzada `kfold`: 

In [33]:
modelos_grid = GridSearchCV(PolynomialRegression(), param_grid, cv=kfold, n_jobs=-1)

Finalmente, entrenaremos todos los modelos usando el método `fit()` con los conjuntos de entrenamiento (**Nota:** el entrenamiento puede tardar algunos minutos):

In [34]:
modelos_grid.fit(x_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
4 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_base.py", line 718, in fit
    self.coef_, self._residues, self.rank_, self.singular_ = 

GridSearchCV(cv=KFold(n_splits=10, random_state=0, shuffle=True),
             estimator=Pipeline(steps=[('polynomialfeatures',
                                        PolynomialFeatures()),
                                       ('linearregression',
                                        LinearRegression())]),
             n_jobs=-1, param_grid={'polynomialfeatures__degree': [2, 3, 4]})

Podemos obtener el mejor valor del hiperparámetro usando el atributo `best_params_`:

In [35]:
print("Mejor parámetro: ", modelos_grid.best_params_)

Mejor parámetro:  {'polynomialfeatures__degree': 2}


Además podemos guardar nuestro mejor modelo, ya entrenado, usando el atributo `best_estimator_`:

In [36]:
mejor_modelo = modelos_grid.best_estimator_

### Evaluación del modelo

Finalmente, realizaremos predicciones sobre el conjunto de pruebas y obtendremos las tres métricas de rendimiento. De la misma forma que con el conjunto de entrenamiento, primero vamos a estandarizar los datos:

In [37]:
x_test = scaler.fit_transform(x_test)
x_test = pd.DataFrame(x_test, columns=columns)

Con el conjunto de pruebas estandarizado, podemos realizar las predicciones. Ten en cuenta que no realizamos la transformación de datos, ya que la función `PolynomialRegression()` se encarga de ello:

In [38]:
y_pred = mejor_modelo.predict(x_test)

print('------ Modelo de regresión polinomial múltiple----')
print("RMSE: %.2f" % mean_squared_error(y_test, y_pred, squared=False))
print("MAE: %.2f" % mean_absolute_error(y_test, y_pred))
print('R²: %.2f' % r2_score(y_test, y_pred))

------ Modelo de regresión polinomial múltiple----
RMSE: 163152.26
MAE: 102681.31
R²: 0.78


Puedes verificar que este modelo es el mismo que el que entrenamos manualmente usando el atributo `degree=2`, pero obtenido después de realizar una búsqueda de hiperparámetros.

## Cierre

En este tutorial hemos utilizado Pandas y scikit-learn para preparar nuestro conjunto de datos, entrenar un modelo de regresión polinomial y evaluar el modelo resultante. Específicamente, comparamos un modelo de regresión lineal simple con el modelo de regresión polinomial, visualizando los cambios realizados al conjunto de datos y comparando sus desempeños. Adicionalmente, realizamos una búsqueda exhaustiva de hiperparámetros usando nuevas clases de scikit-learn.

---
Si quieres más información sobre el objeto `PolynomialFeatures()` de `scikit_learn` puedes consultar el [sitio web oficial](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html)

Para la estandarización de datos con la clase `RobustScaler()` puedes consultar [este enlace](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html)

Para la creación de pipelines o secuencias usando `make_pipeline()` puedes consultar [este enlace](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_pipeline.html)

Para la búsqueda de hiperparámetros con la clase `GridSearchCV()` puedes consultar [este enlace](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

Finalmente, para obtener más información sobre la clase `KFold()` puedes ir [aquí](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html)

---
*Creado por: Nicolás Díaz*  
*Revisado por: Haydemar Nuñez*  
*Versión de: Julio 4, 2022*  
*Universidad de los Andes*  